In [0]:
import pandas as pd
import numpy as np

**Sebelum Lanjut!,   jalankan "Librari Wrapper-Forward" di bagian paling bawah**

In [0]:
#data_cleanU1.xlsx
data = pd.read_excel("https://github.com/lufias69/Spam_Detection/blob/master/Filter%20chi%20square/komentar/data_cleanU1.xlsx?raw=true")
# data = data[:200]
komentar = data.komentar.tolist()
label = data.label.tolist()
 
y = np.array(label)
print(len(y))
 
data

5047


,Column1,label,komentar
0,195,spam,ga nyesel deh aku order sana
1,480,non spam,wakakak tolol tulang tulang
2,539,non spam,sepatu keren kamu keren cara instan
3,564,spam,via q udh blja shoppe barang bagus kecewa mantap
4,596,non spam,sir what if use again kamisenglish that s just...
...,...,...,...
5042,5049,spam,bosen outfit kalian aja yuk cek ig tambah kole...
5043,5050,spam,maantaap banggeet kaak eteaah akuu pakee gemuk...
5044,5052,spam,sllu ajar usaha tahan kualitas costumer neng s...
5045,5053,spam,aasi ane o a uda nenain odu yan sana eui eecay...


**seleksi fitur menggunakan filter**

In [0]:
 
sf = ChiSquare(alpha = 0.1)
#inisialisasi
sf.find_best_features(komentar, label)

In [0]:
print("jumlah fitur terbaik:",len(sf.best_features),"dari",len(sf.fitur), "fitur")
best_features = sf.best_features
sf.data #untuk memunculkan semua data
sf.alpha #untuk mengetahui alpha

jumlah fitur terbaik: 2749 dari 15136 fitur


0.1

**Seleksi fitur Wrapper-forward**

In [0]:
#ini lama bangets
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
hasil = wrapper(model=MultinomialNB(),vocabs = best_features,  weighting=TfidfVectorizer, corpus=komentar, y=y,n_splits=5, target=0.97)
 
best_fitur = hasil[1]
for i in best_fitur:
    print(i)

2749
$.^^.
1| 0.5557758392291162
2| 0.6316651129929644
3| 0.6835725990834961
4| 0.7053665525125357
5| 0.7204286176883297
6| 0.7339031881384372
7| 0.7751152498797946
8| 0.8044443572206577
9| 0.8189080454130646
10| 0.8319857912451305
11| 0.8412988057973291
12| 0.8504126230264255
13| 0.8583379289366002
14| 0.8650735460067315
15| 0.8718103405979845
16| 0.8771608003218558
17| 0.8829067108891266
18| 0.8880597395715786
19| 0.8930145521985301
20| 0.8965820486904985
21| 0.899753309324986
22| 0.9025263715667899
23| 0.9051025915277355
24| 0.9076786152351607
25| 0.9102540501820251
26| 0.9120379946815296
27| 0.9140195664759737
28| 0.9160007457633773
29| 0.9177841015023207
30| 0.9193694374392841
31| 0.9209543808692068
32| 0.9223415007506697
33| 0.9237284243786125
34| 0.9249171319510543
35| 0.9265020753809772
36| 0.9278893915159603
37| 0.9290780990884023
38| 0.9302668066608446
39| 0.9312574944313063
40| 0.9322477896947277
41| 0.9330406539167297
42| 0.9338335181387315
43| 0.9346261861072133
44| 0.9354

**Librari Wrapper-Forward**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
 
from sklearn.naive_bayes import MultinomialNB
 
from sklearn.model_selection import StratifiedKFold
 
def convert_bin(data, key=False):
    new_data = list()
    if (type(data)==list or type(data)==str) and key==False:
        if type(data)==str:
            for d in data:
                new_data.append(int(d))
        else:
            for d in data:
                new_data.append(int(d))
        return new_data
    else:
        for d in data:
            new_data.append(str(d))
        return "".join(new_data)
            
def join_bin(data1, data2):
    new_data = list()
    for a,b in zip(data1, data2):
        if a==b:
            new_data.append(a)
        elif a!=b:
            if b==1:
                new_data.append(b)
            elif a==1:
                new_data.append(a)
            else:
                new_data.append(0)
    return new_data
 
def gen(x, urutan):
    first_bin = [0 for i in range(0,x)]
    first_bin[urutan]=1
    return first_bin
 
def generate_bins(n):
    bin_all = list()
    for i in range(0,n):
        bin_gen = gen(n,i)
        bin_all.append(bin_gen)
    return bin_all
 
def convert_to_index(data):
    new_data = list()
    for idx, i in enumerate(data):
        if i!=0:
            new_data.append(idx)
    return new_data
 
 
 
def wrapper(vocabs, model=None, weighting=TfidfVectorizer(), corpus=None, y=None,n_splits=5, target=0.97):
    
    y=np.array(y)
    vectorizer = weighting(vocabulary = vocabs)
    X = vectorizer.fit_transform(corpus)
    fitur = np.array(vectorizer.get_feature_names())
    panjang_fitur = len(fitur)
 
    print(len(vectorizer.get_feature_names()))
    
    binari_pertama = generate_bins(panjang_fitur)
    scor_binari_pertama = list()
 
    skf = StratifiedKFold(n_splits=n_splits)
    skf_split = skf.split(X, y)
 
 
    print("$",end=".")
    for b in binari_pertama:
        idx_bin = convert_to_index(b)
        # print(fitur[idx_bin],end="")
        vectorizer = weighting(vocabulary=fitur[idx_bin])
        X = vectorizer.fit_transform(corpus)
        
        skf = StratifiedKFold(n_splits=n_splits)
        skor_list = list()
        
        for train_index, test_index in skf.split(X, y):
            # print(X[train_index])
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            model.fit(X_train, y_train)
            skor_list.append(model.score(X_test,y_test))
            
        scor_binari_pertama.append(sum(skor_list)/len(skor_list))
    print("^^",end=".") 
    print("")
    max_index = scor_binari_pertama.index(max(scor_binari_pertama))
    pilihan_pertama_bin = binari_pertama[max_index]
    print(sum(pilihan_pertama_bin), end="| ")
    print(max(scor_binari_pertama))
    
    all_bin = list()
    all_skor = list()
 
    best_fitur = list()
    best_skor = list()
    
    for _ in range(panjang_fitur-1):
        # print(_, end=".")
        new_bin_ = list()
        
        for idx, i in enumerate(binari_pertama):
          if convert_bin(i, key=True) != convert_bin(pilihan_pertama_bin, key=True):
            # if idx !=pilihan_pertama_bin:
            new_bin_.append(join_bin(pilihan_pertama_bin, i))
                
        # print(sum(new_bin_[0]))
        
        binari_pertama = list(new_bin_)
        scor_binari_pertama = list()
        for b in new_bin_:
            idx_bin = convert_to_index(b)
            vectorizer = weighting(vocabulary=fitur[idx_bin])
            X = vectorizer.fit_transform(corpus)
 
            skf = StratifiedKFold(n_splits=n_splits)
            skor_list = list()
            for train_index, test_index in skf.split(X, y):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
 
                model.fit(X_train, y_train)
                skor_list.append(model.score(X_test,y_test))
 
            scor_binari_pertama.append(sum(skor_list)/len(skor_list))
        #
        max_index = scor_binari_pertama.index(max(scor_binari_pertama))
        pilihan_pertama_bin = new_bin_[max_index]
        print(sum(pilihan_pertama_bin), end="| ")
 
        print(max(scor_binari_pertama))
 
        best_fitur.append(fitur[convert_to_index(pilihan_pertama_bin)])
        best_skor.append(max(scor_binari_pertama))
        
        # all_bin += list(new_bin_)
        # all_skor += list(scor_binari_pertama)
        
        if max(scor_binari_pertama) >= target:
            return (max(scor_binari_pertama), fitur[convert_to_index(pilihan_pertama_bin)])
        elif sum(pilihan_pertama_bin)>= panjang_fitur-1:
          return (max(scor_binari_pertama), fitur[convert_to_index(pilihan_pertama_bin)])
        
    max_index = best_skor.index(max(best_skor))
    # pilihan_pertama_bin = new_bin_[max_index]
    best_of_dbest_fitur = best_fitur[max_index]
    return (max(best_skor), best_of_dbest_fitur)

****Filter Library****

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
 
import pandas as pd
 
#list = array (indexnya 0,1,2, dst) tdk menentukan indexnya di awal, dict = hashmap ('a') harus mengisi key nya
def data_separate(y, complement=False):
    kelas = sorted(set(y))
    dict_index = dict()
    for c in kelas:
        index = list()
        for ix, c_ in enumerate(y):
            if complement==False and c==c_:
                index.append(ix)
            elif complement==True and c!=c_:
                index.append(ix)
        dict_index.update({c:index})
    return dict_index
 
class ChiSquare:
    def __init__(self, alpha = 0.001):
        self.alpha = alpha
 
        self.nilai_kritis = {
            0.1   : 2.71,
            0.05 : 3.84,
            0.01 : 6.63,
            0.005 : 7.88,
            0.001 : 10.83
        }
 
        if self.alpha not in self.nilai_kritis:
            print(self.alpha, "tidak termasuk dalam tabel nilai kritis")
 
    def find_best_features(self, data, label, fitur = None):
 
        if fitur == None:
            vectorizer = CountVectorizer(binary=True)
        else:
            vectorizer = CountVectorizer(vocabulary = fitur, binary=True)
 
        X = vectorizer.fit_transform(data)
        nchi = X.shape[0]
        self.fitur = vectorizer.get_feature_names()
        index_doc = data_separate(label)
        index_doc_complement = data_separate(label, complement=True)
 
        self.classes = list(set(label))
 
        self.dict_chis = {"fitur":self.fitur}
        #fitur adalah key, self.fitur (isinya) berasal dr ln 44 mengambil semua fitur
        # for c_ in list(reversed(self.classes)):
        for c_ in self.classes:
            A = np.sum(X[index_doc[c_]], axis=0).A
            B = X[index_doc[c_]].shape[0] - A
 
            C = np.sum(X[index_doc_complement[c_]], axis=0).A
            D = X[index_doc_complement[c_]].shape[0] - C
 
            AD = A*D
            CB = C*B
 
            atas = nchi * (AD - CB)**2
            bawah = (A + C) * (B + D) * (A + B) * (C + D)
            x_pangkat2 = atas/bawah
            # print(atas, bawah)
            self.dict_chis.update({c_:list(x_pangkat2[0])})
            #memasukkan hasil perhitungan untuk setiap kelas disimpan dalam dict dimana key nya adalah kelasnya (c_)
            #kalo di pyton berbasis objec, object akan bicara memory
            # print(A, B, C, D, "| ")
            # print("=================================================")
        self.best_features = list()
        nk = self.nilai_kritis[self.alpha] #Ini adalah implemenasi dari dictionary
        # print(self.dict_chis[self.classes[0]])
        for term, sp, nsp in zip(self.fitur, self.dict_chis[self.classes[0]], self.dict_chis[self.classes[1]]):
            #akan dicek satu per satu fitur
            #zip --> menyatukan beberapa list atau array, classes[0] adalah spam -> mengambil data chisquare utk kelas spam, classes[1] adalah nonspam
            # print(sp, nsp)
            if sp>=nk or nsp>=nk:
                self.best_features.append(term)
 
        self.data = pd.DataFrame.from_dict(self.dict_chis)